# TP 12 Simulación - Call Center Metrogas

## Contexto
Se dispone de puestos de emergencia con una cola.
Se busca estudiar la cantidad más óptima de la cantidad de puestos que cumpla con el PIC.

## Importando librerias de simpy

In [1]:
#from simpy import initialize
#from SimPy.Simulation import *
import simpy
import random
import collections
from itertools import combinations_with_replacement
import xlwt 
from xlwt import Workbook 
import time

_____

## Simulacion de una llegada

In [2]:
def llegada(env,puestos,cont):
    
    while True:
        
        cont["NT"] += 1
            
        ce = Customer(name = "clienteE%02d"%cont["NT"])
            
        # Genero IA para el siguiente cliente y genero el timeout para su llegada 
        ia = generar_IA(cont)
        yield env.timeout(ia)
        cont['file'].write("T=%5.2f %s: COLA EMERGENCIA - llegue con un IA: %5.2f \n"%(env.now,ce.name,ia))
        colas(puestos,cont) # Imprimo el estao de las colas
            
        # Genero la llamada
        env.process(ce.llamada_emergencia(env,puestos,cont))
            

In [3]:
class Customer():
    """Evento llegadas/llamadas de los clientes"""
    
    def __init__(self,name):
        self.name = name
        
    def llamada_emergencia(self,env,puestos,cont):
        
        arrive = env.now
        # Pido uso de un puesto
        emergencia = puestos.request()
        atendido = yield emergencia
        wait = env.now-arrive #Tiempo de espera
            
        if wait <= 40:
            cont["NEC"] += 1
            
        ta = generar_TA(cont)
        cont['file'].write("T=%5.2f, %s: COLA EMERGENCIA - Me atendieron en un puesto de emergencia despues de esperar %5.2f \n"%(env.now,self.name,wait))
        colas(puestos,cont)
        cont['file'].write("Tiempo de Atencion generado para %s: %5.2f \n"%(self.name,ta))
        yield env.timeout(ta)
        cont['file'].write("T=%5.2f, %s: Sali de un puesto de emergencia \n"%(env.now,self.name))
            

In [4]:
def colas(p,cont):
    cont['file'].write("Atendiendo: %1d \n"%(p.count))
    cont['file'].write("En cola: %1d \n"%(len(p.queue)))


In [69]:
def generar_TA(cont):
    r = 0

    # Mientras caiga fuera de esta zona, generame otro
    while r < 2 or r > 126:

        try:
            r = int(random.lognormvariate(muTA, sigmaTA))
        except OverflowError:
            r = float('inf')
    
    return r

In [70]:
def generar_IA(cont):
    rIA = 0

    while rIA <= 2 or rIA > 255:

        try:
            rIA = int(random.lognormvariate(muIA, sigmaIA))
        except OverflowError:
            rIA = float('inf')
    
    return rIA

## Inicializacion de la simulacion

In [14]:
def simular(cantEmer,ci,maxTime):
    env = simpy.Environment()
    puestos = simpy.Resource(env,capacity=cantEmer) 
           
    env.process(llegada(env,puestos,ci))
    env.run(until=maxTime)
    

## Condiciones Iniciales

In [73]:
# Experimental data
NT = 0
maxTime = 1000 #segundos
NEC = 0 # Cantidad de gente que espero en cola menor a 40 segundos
PEC = 0.0 # Porcentaje de espera en cola menor a 40 segundos
file = None

# TA - distribucion normal
muTA = 128.857
sigmaTA = 126.335

# IA - distribucion normal
muIA = 8.817
sigmaIA = 5813.745

# Contadores
ci = {"muTA":muTA,
     "sigmaTA":sigmaTA,
     "muIA":muIA,
     "sigmaIA":sigmaIA,
     "emer":0,
     "NEC":NEC,
     "NT":NT,
     "file":file}

## Ingreso manual de Puestos de atencion - variables de control

!! Generar una version para automatizar esto, que genere diferentes combinaciones dado un rango

Y tmbn dejar la opcion de generarlo manualmente


print('Ingrese cantidad de puestos comerciales')
cantCom = int(input())
ci["com"] = cantCom
print('Ingrese cantidad de puestos de emergencia')
cantEmer = int(input())
ci["emer"] = cantEmer
print('Ingrese cantidad de puestos polifuncionales')
cantPolif = int(input())
ci["pol"] = cantPolif

simular(cantCom,cantEmer,cantPolif,ci,maxTime)


## Ejecucion automatica con combinaciones de variables de control

In [75]:
# Create an Excel to record the results 
wb = Workbook() 
t = time.localtime()
current_time = time.strftime("%Hh%Mm", t)
sheet = wb.add_sheet(str(current_time)) 
sheet.write(0, 0, 'Emergencia')
sheet.write(0, 1, 'NT')
sheet.write(0, 2, 'NEC')
sheet.write(0, 3, 'PEC')
simulacion = 0

# Create a text file to record processing log
file = open("log_sim1.txt","w")
ci['file'] = file

cant_puestos = [5,10,15,20,25,30,35,40,45,50]

for i in cant_puestos:
    
    simular(i,ci,maxTime)
    simulacion += 1
        
    sheet.write(simulacion, 0, i)
    sheet.write(simulacion, 1, ci["NT"]) # Cantidad de gente total
    sheet.write(simulacion, 2, ci["NEC"])
    PEC = (ci["NEC"] / ci["NT"]) * 100
    sheet.write(simulacion, 3, PEC) # Cantidad de gente que haya esperado menor a 40 segundos
    file.write("================================================================================ \n") 
    file.write("\n")
        
wb.save('resultados_Simulacion1.xls')
file.close()
print('OK')        

OK
